In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from visualization_helper import *

In [ ]:
import numpy as np

In [ ]:
input_path = '../../data/sphere_cap_0.3.json'
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
import mesh
target_mesh = mesh.Mesh(input_data['target_v'], input_data['target_f'])

### Initialization

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
thickness = io.material_params[6]
curr_um.targetDeploymentHeight = thickness

In [ ]:
rod_colors = []
for ri in range(curr_um.numSegments()):
    rod_colors.append(np.ones(curr_um.segment(ri).rod.numVertices()) * ri)

In [ ]:
import mesh
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
# view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
# view.update(scalarField = rod_colors)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
        dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-6
curr_um.setDoFs(dof)

In [ ]:
curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.LinearActuator

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e-3
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 0.001
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.5)
fixedVars = curr_um.rigidJointAngleDoFIndices()

In [ ]:
curr_um.energyElastic(), curr_um.energyDeployment(), curr_um.energyAttraction()

In [ ]:
view.showScalarField(rod_colors)

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False

In [ ]:
def eqm_callback(prob, i):
    view.showScalarField(rod_colors)

In [ ]:
result = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = curr_um.rigidJointAngleDoFIndices())

In [ ]:
result.success

In [ ]:
import finite_diff

In [ ]:
import importlib
importlib.reload(finite_diff)

In [ ]:
curr_um.setDoFs(curr_um.getDoFs() + np.random.uniform(-0.1, 0.1, curr_um.numDoF()))

In [ ]:
view.update()

In [ ]:
direction = np.random.uniform(-1, 1, curr_um.numDoF())

In [ ]:
curr_um.updateSourceFrame()

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1

In [ ]:
finite_diff.gradient_convergence_plot(curr_um, etype=elastic_rods.EnergyType.Full, direction = direction)

In [ ]:
finite_diff.hessian_convergence_plot(curr_um, etype=elastic_rods.EnergyType.Full)